In [1]:
import re
import sys
from operator import add

import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
def compute_contributions(urls, rank):
    total_urls = len(urls)
    for url in urls:
        yield (url, rank / total_urls)

def parse_neighbors(urls):
    parts = re.split(r'\s+', urls)
    print((parts[0], parts[1]))
    return parts[0], parts[1]

In [2]:
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')

In [3]:
spark = SparkSession.builder\
        .appName('Spark Page Rank Test')\
        .master('spark://172.31.82.177:7077')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 02:42:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
rank_check = spark.read.load(
    'hdfs://172.31.82.177:9000/page_rank_partitioned.csv',
    format = 'csv',
    inferSchema = 'true',
    sep = ',',
    header = 'true'
)

In [5]:
rank_check.sort(col('rank').desc()).show(10)

+------+------------------+
|  link|              rank|
+------+------------------+
|272919| 6531.324623752435|
|438238| 4335.323158564443|
|571448|  2383.89760741189|
|601656|2195.3940755967287|
|316792|1855.6908757901415|
|319209|1632.8193684975688|
|184094| 1532.284237448332|
|571447|1492.9301630938783|
|401873| 1436.160093346926|
| 66244|1261.5783958673344|
+------+------------------+
only showing top 10 rows



In [11]:
rank_check.createOrReplaceTempView('rank_temp')

spark.sql('SELECT COUNT(1) FROM rank_temp').show()

+--------+
|count(1)|
+--------+
|  596668|
+--------+



In [5]:
lines = spark.read.text(
    'hdfs://172.31.82.177:9000/web-BerkStan.txt'
).rdd.map(lambda r: r[0])

In [36]:
# lines.show(5)

In [35]:
# filtered_lines = lines.filter(~(lines.value.contains('#')))

In [8]:
# filtered_lines.show()

+--------------+
|         value|
+--------------+
|          1\t2|
|          1\t5|
|          1\t7|
|          1\t8|
|          1\t9|
|         1\t11|
|         1\t17|
|     1\t254913|
|     1\t438238|
|254913\t255378|
|254913\t255379|
|254913\t255383|
|254913\t255384|
|254913\t255392|
|254913\t255393|
|254913\t255394|
|254913\t255396|
|254913\t255399|
|254913\t255401|
|254913\t255402|
+--------------+
only showing top 20 rows



In [18]:
parse_neighbors('1\t2')

('1', '2')


('1', '2')

In [6]:
links = lines.map(lambda urls: parse_neighbors(urls)).distinct().groupByKey()

In [26]:
links.take(10)

[('121', <pyspark.resultiterable.ResultIterable at 0x7fa50a6ae8c0>),
 ('4801', <pyspark.resultiterable.ResultIterable at 0x7fa50a6ae9b0>),
 ('10866', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aead0>),
 ('18115', <pyspark.resultiterable.ResultIterable at 0x7fa50a6ae830>),
 ('18743', <pyspark.resultiterable.ResultIterable at 0x7fa50a6ae9e0>),
 ('24038', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aec20>),
 ('24473', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aeb00>),
 ('31513', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aed10>),
 ('36635', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aecb0>),
 ('43855', <pyspark.resultiterable.ResultIterable at 0x7fa50a6aed70>)]

Exception in thread "serve RDD 38 with partitions 0" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [7]:
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [8]:
for iteration in range(10):
    contribs = links.join(ranks).flatMap(
        lambda url_urls_rank: compute_contributions(url_urls_rank[1][0], url_urls_rank[1][1])
    )

    # Update each page’s rank to be 0.15 + 0.85 * (sum of contributions)
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

In [31]:
# for (link, rank) in ranks.collect():
#     print("%s has rank: %s." % (link, rank))

In [9]:
column_names = ['link', 'rank']
ranks_df = ranks.toDF(column_names)

23/02/18 19:43:57 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.31.82.177 executor 0): org.apache.spark.sql.execution.QueryExecutionException: Encountered error while reading file hdfs://172.31.82.177:9000/web-BerkStan.txt. Details: 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:731)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:283)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.ContextAwareIterator.hasNext(ContextAwareIterator.scala:39)
	at org.apache.spark.api.python.SerDeUtil$AutoBatchedPickler.hasNext(SerDeUtil.scala:86)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.a

23/02/18 19:44:16 WARN TaskSetManager: Lost task 1.1 in stage 0.0 (TID 3) (172.31.82.177 executor 0): org.apache.hadoop.hdfs.BlockMissingException: Could not obtain block: BP-1929799945-172.31.82.177-1676508760369:blk_1073741830_1006 file=/web-BerkStan.txt No live nodes contain current block Block locations: Dead nodes: 
	at org.apache.hadoop.hdfs.DFSInputStream.refetchLocations(DFSInputStream.java:1007)
	at org.apache.hadoop.hdfs.DFSInputStream.chooseDataNode(DFSInputStream.java:990)
	at org.apache.hadoop.hdfs.DFSInputStream.chooseDataNode(DFSInputStream.java:969)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:677)
	at org.apache.hadoop.hdfs.DFSInputStream.readWithStrategy(DFSInputStream.java:884)
	at org.apache.hadoop.hdfs.DFSInputStream.read(DFSInputStream.java:957)
	at java.io.DataInputStream.read(DataInputStream.java:149)
	at org.apache.hadoop.mapreduce.lib.input.UncompressedSplitLineReader.fillBuffer(UncompressedSplitLineReader.java:62)
	at org.apache.h

23/02/18 19:44:46 WARN TaskSetManager: Lost task 0.3 in stage 0.0 (TID 6) (172.31.82.177 executor 0): org.apache.spark.sql.execution.QueryExecutionException: Encountered error while reading file hdfs://172.31.82.177:9000/web-BerkStan.txt. Details: 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:731)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:283)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.ContextAwareIterator.hasNext(ContextAwareIterator.scala:39)
	at org.apache.spark.api.python.SerDeUtil$AutoBatchedPickler.hasNext(SerDeUtil.scala:86)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.a

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 6) (172.31.82.177 executor 0): org.apache.spark.sql.execution.QueryExecutionException: Encountered error while reading file hdfs://172.31.82.177:9000/web-BerkStan.txt. Details: 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:731)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:283)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.ContextAwareIterator.hasNext(ContextAwareIterator.scala:39)
	at org.apache.spark.api.python.SerDeUtil$Auto

23/02/18 19:44:49 WARN TaskSetManager: Lost task 1.3 in stage 0.0 (TID 7) (172.31.82.177 executor 0): TaskKilled (Stage cancelled)


In [10]:
ranks_df.show()

NameError: name 'ranks_df' is not defined

23/02/18 19:52:00 ERROR TaskSchedulerImpl: Lost executor 0 on 172.31.82.177: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/02/18 19:52:01 WARN StandaloneAppClient$ClientEndpoint: Connection to 172.31.82.177:7077 failed; waiting for master to reconnect...
23/02/18 19:52:01 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
23/02/18 19:52:01 WARN StandaloneAppClient$ClientEndpoint: Connection to 172.31.82.177:7077 failed; waiting for master to reconnect...


In [6]:
spark.stop()

23/02/20 02:43:31 WARN NioEventLoop: Selector.select() returned prematurely 512 times in a row; rebuilding Selector io.netty.channel.nio.SelectedSelectionKeySetSelector@4b2a03c4.
